In [1]:
import os
# initialize openai here

In [2]:
import datetime
from typing import Literal, Optional, Tuple

from langchain_core.pydantic_v1 import BaseModel, Field

class SubQuery1(BaseModel):
    """Search over a database of tutorial videos about a software library."""

    sub_query: str = Field(
        ...,
        description="A very specific query against the database.",
    )
class SubQuery2(BaseModel):
    """Search over a database of ecommerce website about a clothing product."""

    sub_query: str = Field(
        ...,
        description="A very specific query against the database.",
    )

In [3]:
from langchain.output_parsers import PydanticToolsParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

system = """You are an expert at converting user questions into database queries. \
You have access to a database of tutorial videos  about a software library and ecommerce website about the clothing and products. \

Perform query decomposition. Given a user question, break it down into distinct sub questions that \
you need to answer in order to answer the original question.

If there are acronyms or words you are not familiar with, do not try to rephrase them."""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
llm_with_tools = llm.bind_tools([SubQuery1, SubQuery2])
parser = PydanticToolsParser(tools=[SubQuery1, SubQuery2])
query_analyzer = prompt | llm_with_tools | parser

In [7]:
query_analyzer.invoke(
    {
        "question": "get the iphone7"
    }
)

[SubQuery1(sub_query='tutorial videos about iPhone 7 features'),
 SubQuery2(sub_query='iPhone 7 availability on ecommerce websites')]